In [26]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
import data_retrieval.psql_methods as psql
import pandas as pd
import snob_utils as snob
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
from scipy import stats
import statistics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
command = "SELECT * from collection_total_stats"
temp_stats_tuples = psql.execute_commands([command])
columns = ['volume','sales', 'avg_price','num_owners','market_cap','floor','symbol','slug']
stats_df = pd.DataFrame(temp_stats_tuples,columns=columns)

In [7]:
stats_df.sort_values(by='volume',ascending=False,inplace=True)
stats_df.head(20)
#get slugs from top 20 stats_df
slugs = list(stats_df['slug'].head(20).values)
slugs

['boredapeyachtclub',
 'mutant-ape-yacht-club',
 'azuki',
 'otherdeed',
 'art-blocks',
 'clonex',
 'proof-moonbirds',
 'doodles-official',
 'bored-ape-kennel-club',
 'pudgypenguins',
 'meebits',
 'beanzofficial',
 'degods',
 'cool-cats-nft',
 'sandbox',
 'milady',
 'rarible',
 'ens',
 '0n1-force',
 'world-of-women-nft']

In [4]:
df = snob.create_master_snob_df()

Dataset ReturnIndexDataset
    Number of datapoints: 373074
    Root location: /global/scratch/tlundy/NFT_Research/nft_research/Dino/images_features/expanded_images/val


100%|██████████| 1184/1184 [00:02<00:00, 411.96it/s]


Dataset ReturnIndexDataset
    Number of datapoints: 846346
    Root location: /global/scratch/tlundy/NFT_Research/nft_research/Dino/images_features/images/val


100%|██████████| 1265/1265 [00:03<00:00, 409.25it/s]


In [15]:
#find out how many slugs from list are in amster df as collections
collections = df[df['Collection'].isin(slugs)]
#count unique collections
collection_slugs = list(collections['Collection'].unique())

In [23]:
#Check whether there is significant correlation between sale price and distance for each slug in collection_slugs
valid_slugs = []
for slug in collection_slugs:
    slug_df  = collections[collections['Collection']==slug]
    distance  = slug_df['distance']
    price = slug_df['sale_price']
    rarity = slug_df['rarity_rank']
    correlation_1, p_value_1 = pearsonr(distance, price)
    correlation_2, p_value_2 = pearsonr(rarity, price)
    if p_value_1 < 0.05 or p_value_2 < 0.05:
        print('Rarity',slug,correlation_2, p_value_2)
        print('Distance',slug,correlation_1, p_value_1)
        valid_slugs.append(slug)
valid_tuple = tuple(valid_slugs)


Rarity cool-cats-nft -0.059736070962043364 0.17131496318167755
Distance cool-cats-nft 0.11246587377218356 0.0098392510091859
Rarity proof-moonbirds -0.2296776586945044 1.5704389265927143e-06
Distance proof-moonbirds 0.29641243900741115 3.964478810405878e-10
Rarity beanzofficial -0.1855303764736122 0.00010697046338900099
Distance beanzofficial 0.24392743317006213 2.9438617857540577e-07
Rarity mutant-ape-yacht-club -0.256510560897773 2.016094468436212e-08
Distance mutant-ape-yacht-club -0.05060088560993747 0.2761930544814938
Rarity meebits -0.12570333101321443 0.027647083651857057
Distance meebits 0.09489872725113846 0.09696983375363562
Rarity doodles-official -0.25403087704024674 3.096877276279076e-09
Distance doodles-official 0.33360551785713566 3.230033971335355e-15
Rarity pudgypenguins -0.11538271050716765 0.005089881347500054
Distance pudgypenguins -0.06049686749847332 0.14286862702298575
Rarity azuki -0.19354029040955031 4.644521920144554e-06
Distance azuki 0.031085748988013036 0.4

In [27]:
valid_tuple
command = [f"select * from nfttoimage where slug in {valid_tuple}"]
rows  = psql.execute_commands(command)

In [31]:
# Column names for the DataFrame
columns = ['slug', 'token_id', 'url']

# Create a DataFrame from the list of tuples
df_images = pd.DataFrame(rows, columns=columns)
#group by slug and count number of images
df_images.groupby('slug').count()


,token_id,url
slug,,
azuki,10000,10000
beanzofficial,19950,19950
clonex,19512,19512
cool-cats-nft,9965,9965
doodles-official,10000,10000
meebits,20000,20000
mutant-ape-yacht-club,19483,19483
proof-moonbirds,10000,10000
pudgypenguins,8888,8888


In [1]:
import pickle
#dump valid slugs to pickle
# with open('valid_slugs.pkl', 'wb') as f:
#     pickle.dump(valid_tuple, f)
#read valid slugs from pickle
with open('valid_slugs.pkl', 'rb') as f:
    valid_tuple = pickle.load(f)

In [2]:
valid_tuple

('cool-cats-nft',
 'proof-moonbirds',
 'beanzofficial',
 'mutant-ape-yacht-club',
 'meebits',
 'doodles-official',
 'pudgypenguins',
 'azuki',
 'clonex')

After this point create_case_study_images() was called, and then the resulting images had their features exrtacted using feature_extract_notebook

In [4]:
command = f"SELECT slug,token_id,sale_price from cf_sales where slug in {valid_tuple}"
temp_stats_tuples = psql.execute_commands([command])
columns = ['slug','token_id', 'sale_price']
sales_df = pd.DataFrame(temp_stats_tuples,columns=columns)

In [7]:
#take sales df and group by slug and token_id, averaging sale price
avg_sales = sales_df.groupby(['slug','token_id']).mean().reset_index()
avg_sales.sort_values(by=['slug','token_id'],ascending=False,inplace=True)

In [8]:
avg_sales

,slug,token_id,sale_price
95299,pudgypenguins,8887,0.805633
95298,pudgypenguins,8886,9.105373
95297,pudgypenguins,8885,0.690000
95296,pudgypenguins,8884,4.059311
95295,pudgypenguins,8883,1.962500
...,...,...,...
4,azuki,58,15.722500
3,azuki,53,11.226680
2,azuki,52,8.750000
1,azuki,49,5.665000


In [10]:
df = snob.assemble_data_and_features('snob_case_study','snob_case_study_features')
df

Dataset ReturnIndexDataset
    Number of datapoints: 137794
    Root location: /global/scratch/tlundy/NFT_Research/nft_research/Dino/images_features/snob_case_study/val


,Label,Features,Collection,NFT_num,Contract
0,0,"[0.0013048029504716396, -0.013092035427689552,...",azuki,0,0xed5af388653567af2f388e6224dc7c4b3241c544
1,0,"[0.027294615283608437, -0.04734836518764496, -...",azuki,1,0xed5af388653567af2f388e6224dc7c4b3241c544
2,0,"[0.03676558658480644, -0.018740998581051826, -...",azuki,10,0xed5af388653567af2f388e6224dc7c4b3241c544
3,0,"[-0.004882593639194965, -0.011051790788769722,...",azuki,100,0xed5af388653567af2f388e6224dc7c4b3241c544
4,0,"[-0.016948213800787926, 0.00456386199221015, 0...",azuki,1000,0xed5af388653567af2f388e6224dc7c4b3241c544
...,...,...,...,...,...
137789,9,"[-0.09531647711992264, -0.007036624941974878, ...",pudgypenguins,995,0xbd3531da5cf5857e7cfaa92426877b022e612cf8
137790,9,"[-0.046095795929431915, -0.01598743535578251, ...",pudgypenguins,996,0xbd3531da5cf5857e7cfaa92426877b022e612cf8
137791,9,"[-0.06151833385229111, -0.02195701375603676, -...",pudgypenguins,997,0xbd3531da5cf5857e7cfaa92426877b022e612cf8
137792,9,"[-0.04027863219380379, -0.03682368993759155, -...",pudgypenguins,998,0xbd3531da5cf5857e7cfaa92426877b022e612cf8


In [16]:
snob.get_nft_rarities(df)

28


  0%|          | 0/5000 [00:00<?, ?it/s]

0x1a92f7381b9f03921564a437210bb9396471050c 0


  0%|          | 2/5000 [00:00<16:09,  5.16it/s]

0x1a92f7381b9f03921564a437210bb9396471050c 1
0x1a92f7381b9f03921564a437210bb9396471050c 10


...

0x1a92f7381b9f03921564a437210bb9396471050c 1072
0x1a92f7381b9f03921564a437210bb9396471050c 1073


  2%|▏         | 86/5000 [00:17<16:07,  5.08it/s]

0x1a92f7381b9f03921564a437210bb9396471050c 1074
0x1a92f7381b9f03921564a437210bb9396471050c 1075


  2%|▏         | 86/5000 [00:17<16:46,  4.88it/s]


KeyboardInterrupt: 

In [24]:
df = snob.assemble_data_and_features('snob_case_study','snob_case_study_features')
selected_columns = sorted(df[['Contract', 'NFT_num']].to_numpy().tolist())
command = "Select contract,token_id from nft_to_rarity_2"
rarities = psql.execute_commands([command])
print(len(selected_columns))
selected_columns_trim = [tuple(x) for x in selected_columns]
rarities_set = set(rarities)
l1 = [list(sublist) for sublist in selected_columns_trim if sublist not in rarities_set]
print(len(l1))

Dataset ReturnIndexDataset
    Number of datapoints: 137794
    Root location: /global/scratch/tlundy/NFT_Research/nft_research/Dino/images_features/snob_case_study/val
137794
131863


In [30]:
snob.get_nft_rarities(df)


137794
131863
